<a href="https://colab.research.google.com/github/conuko/AI-Guild/blob/main/experiments/experiment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-Net Experiment 3: Training an U-Net on the entire dataset

The goal of this notebook is to evaluate whether tile-overlapping has an effect on the resulting model.
We will use the exact same setup as in experiment 1 except for the dataset. Each tile has a size of (256, 256) and the step size is 256 to exclude overlap.

All previous experiments have only been used on a small part of the entire dataset. In this experiment all 16 images of
the entire dataset will be split into tiles and used as input for our model.

### 0. Helper Classes

In [ ]:
import numpy as np
from tensorflow import keras
import pickle


class EvaluationMetrics:
    """
        This class calculates and summarizes evaluation metrics based on the predicted and true labels.
    """

    def __init__(self, x_train, x_val, x_test, y_train, y_val, y_test, y_pred, dataset, training_dates, validation_dates, testing_dates, tile_size, step_size,
                 run_count):
       
        self.y_pred = y_pred
        if dataset == 'training':
          self.y_true = y_train
        elif dataset == 'validation':
          self.y_true = y_val
        elif dataset == 'testing':
          self.y_true = y_test
        else: 
          raise Exception(f'Specify the dataset to be used for calculating the metrics. The string has to be either "testing", "training" or "validation", however it was {dataset}.')
        
        self.class_statistics = self.get_statistics(x_train, x_val, x_test, y_train, y_val, y_test)

        self.training_dates = training_dates
        self.validation_dates = validation_dates
        self.testing_dates = testing_dates
        self.tile_size = tile_size
        self.step_size = step_size
        self.run_count = run_count

        self.jacard = self.jacard_coef(self.y_true, y_pred)



        self.conf_matrix_land = self.confusion_matrix(self.y_true, y_pred, 2)
        self.conf_matrix_valid = self.confusion_matrix(self.y_true, y_pred, 1)
        self.conf_matrix_invalid = self.confusion_matrix(self.y_true, y_pred, 0)

        self.precision_land = self.precision(self.conf_matrix_land)
        self.sensitivity_recall_land = self.sensitivity_recall(self.conf_matrix_land)
        self.specificy_land = self.specificy(self.conf_matrix_land)

        self.precision_valid = self.precision(self.conf_matrix_valid)
        self.sensitivity_recall_valid = self.sensitivity_recall(self.conf_matrix_valid)
        self.specificy_valid = self.specificy(self.conf_matrix_valid)

        self.precision_invalid = self.precision(self.conf_matrix_invalid)
        self.sensitivity_recall_invalid = self.sensitivity_recall(self.conf_matrix_invalid)
        self.specificy_invalid = self.specificy(self.conf_matrix_invalid)

        self.f1_land = self.f1_scores(self.conf_matrix_land)
        self.f1_invalid = self.f1_scores(self.conf_matrix_invalid)
        self.f1_valid = self.f1_scores(self.conf_matrix_valid)

    def jacard_coef(self, y_true, y_pred):
        y_true_f = keras.backend.flatten(y_true)
        y_pred_f = keras.backend.flatten(y_pred)

        intersection = keras.backend.sum(y_true_f * y_pred_f)
        return (intersection + 1.0) / (
                keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) - intersection + 1.0
        )  #todo reason for +1?

    def jacard_rounding_issue(self, y_true, y_pred):
        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))
        # one hot encoding
        one_hot_true = np.eye(3)[flatten_true]
        one_hot_pred = np.eye(3)[flatten_pred]
        # calculate intersection (A geschnitten B)
        intersection = np.sum(one_hot_true * one_hot_pred)
        # calculate union (a u B, A vereint B)
        union = len(one_hot_true) + len(one_hot_pred) - intersection
        # return jacard coefficient
        return (intersection + 1) / (union + 1)

    def confusion_matrix(self, y_true, y_pred, label):
        true_positives = 0
        false_positives = 0
        true_negatives = 0
        false_negatives = 0

        # revert one hot encoding => binary tensor [0, 0, 1] back to label [2] (3D array to 2D array)
        label_map_true = np.argmax(y_true, axis=-1)
        label_map_pred = np.argmax(y_pred, axis=-1)
        # convert 2D array into 1D array
        flatten_true = np.reshape(label_map_true, (-1,))
        flatten_pred = np.reshape(label_map_pred, (-1,))

        tp_mask = (flatten_true == flatten_pred) & (flatten_true == label)
        true_positives = np.count_nonzero(tp_mask)

        fn_mask = (flatten_true == label) & (flatten_pred != label)
        false_negatives = np.count_nonzero(fn_mask)

        fp_mask = (flatten_true != label) & (flatten_pred == label)
        false_positives = np.count_nonzero(fp_mask)

        tn_mask = (flatten_true != label) & (flatten_pred != label)
        true_negatives = np.count_nonzero(tn_mask)

        return {
            'true_positives': true_positives,
            'false_positives': false_positives,
            'true_negatives': true_negatives,
            'false_negatives': false_negatives
        }

    def precision(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_positives'])

    def sensitivity_recall(self, conf_matrix):
        return conf_matrix['true_positives'] / (conf_matrix['true_positives'] + conf_matrix['false_negatives'])

    def negative_predictive(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_negatives'])

    def specificy(self, conf_matrix):
        return conf_matrix['true_negatives'] / (conf_matrix['true_negatives'] + conf_matrix['false_positives'])

    def f1_scores(self, conf_matrix):
        prec = self.precision(conf_matrix)
        recall = self.sensitivity_recall(conf_matrix)
        return 2 * prec * recall / (prec + recall)

    def print_metrics(self):
        print(f'jacard index: {self.jacard} \n')

        print(f'precision_land: {self.precision_land}')
        print(f'precision_valid: {self.precision_valid}')
        print(f'precision_invalid: {self.precision_invalid} \n')

        print(f'recall_invalid_land: {self.sensitivity_recall_land}')
        print(f'recall_invalid_land: {self.sensitivity_recall_valid}')
        print(f'recall_invalid_land: {self.sensitivity_recall_invalid} \n')

        print(f'specificy_invalid_land: {self.specificy_land}')
        print(f'specificy_invalid_valid: {self.specificy_valid}')
        print(f'specificy_invalid_invalid: {self.specificy_invalid} \n')

        print(f'f1_land: {self.f1_land}')
        print(f'f1_invalid: {self.f1_invalid}')
        print(f'f1_valid: {self.f1_valid}')

        print(f'Training dates: {self.training_dates}, validation dates: {self.validation_dates}, testing dates: {self.testing_dates}')
        print(f'Number of run: {self.run_count}, tile_size: {self.tile_size}, step_size: {self.step_size}')

    def save_to_file(self):
        file_name = f'../metrics/{self.tile_size}_{self.step_size}_{self.run_count}.pkl'
        with open(file_name, 'wb') as file:
            pickle.dump(self, file)

    def get_label_count(self, array):
        revert_one_hot = np.argmax(array, (-1))
        flatten = np.reshape(revert_one_hot, (-1))
        unique_vals, counts = np.unique(flatten, return_counts=True)
        label_count = {}
        for val, count in zip(unique_vals, counts):
            label_count[f'{val}'] = count
        return label_count

    def get_statistics(self, x_train, x_val, x_test, y_train, y_val, y_test):
       return {'y_train': self.get_label_count(y_train),
                 'y_val': self.get_label_count(y_val), 'y_test': self.get_label_count(y_test)}
    # todo add pixel accuracy

### 1. Loading + Preparing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

drive  sample_data
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  evaluation  notebooks     README.md
Copy_of_unet.ipynb   models	 prepare_data  requirements.txt


In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import (
    Input,
    Conv2D,
    MaxPooling2D,
    concatenate,
    Conv2DTranspose,
    Dropout,
    UpSampling2D
)
from keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
data_directory = "../data_colab/256_256"

x_input = []
y_mask = []

count = 0

for file_name in os.listdir(data_directory):
  file_path = f'{data_directory}/{file_name}'
  print(f'Loading array {file_path}')
  numpy_array = np.load(file_path) 
  y_temp  = numpy_array['y_mask']
  x_temp  = numpy_array['x_input']

  if count == 0:
    x_input = x_temp
    y_mask = y_temp
  else:
    x_input = np.concatenate((x_input, x_temp), axis=0)
    y_mask = np.concatenate((y_mask, y_temp), axis=0)
  count += 1
print(f'Amount of images used: {count}')
print(x_input.shape)
print(y_mask.shape)




Loading array ../data_colab/256_256/2022_10_13.npz
Loading array ../data_colab/256_256/2022_07_15.npz
Loading array ../data_colab/256_256/2022_09_18.npz
Loading array ../data_colab/256_256/2022_06_20.npz
Loading array ../data_colab/256_256/2022_10_23.npz
Loading array ../data_colab/256_256/2022_07_25.npz
Loading array ../data_colab/256_256/2022_08_04.npz
Loading array ../data_colab/256_256/2022_07_10.npz
Loading array ../data_colab/256_256/2022_07_30.npz
Loading array ../data_colab/256_256/2022_08_14.npz
Loading array ../data_colab/256_256/2022_08_24.npz
Loading array ../data_colab/256_256/2022_09_03.npz
Loading array ../data_colab/256_256/2022_12_12.npz
Loading array ../data_colab/256_256/2022_09_08.npz
Loading array ../data_colab/256_256/2022_12_02.npz


Use data generators: Instead of loading all your data into memory at once, use data generators to load the data in batches during training. This can help reduce memory usage. data generators can also split on the fly

In [ ]:
import numpy as np
import keras

# save the entire dataset in one .npz file


class CustomDataGenerator(keras.utils.Sequence):
    def __init__(self, data_path, batch_size, validation_split=0.2, test_split=0.1, shuffle=True):
        self.data_path = data_path
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.test_split = test_split
        self.shuffle = shuffle
        
        # Load the input and target data from the .npz file
        self.data = np.load(self.data_path)
        self.x = self.data['x_input']
        self.y = self.data['y_mask']
        self.num_samples = self.x.shape[0]
        
        # Split the data into training, validation, and test sets
        self.num_train = int(self.num_samples * (1 - validation_split - test_split))
        self.num_val = int(self.num_samples * validation_split)
        self.num_test = self.num_samples - self.num_train - self.num_val
        
        # initializes an array of indices representing the samples in the dataset
        self.indices = np.arange(self.num_samples)
        if shuffle:
            np.random.shuffle(self.indices)
        
    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # Load the input and target data for the batch from the arrays
        batch_x = self.x[batch_indices]
        batch_y = self.y[batch_indices]
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def get_validation_data(self):
        val_indices = self.indices[:self.num_val]
        
        # Load the validation input and target data from the arrays
        val_x = self.x[val_indices]
        val_y = self.y[val_indices]
        
        return val_x, val_y
    
    def get_test_data(self):
        test_indices = self.indices[self.num_train + self.num_val:]
        
        # Load the test input and target data from the arrays
        test_x = self.x[test_indices]
        test_y = self.y[test_indices]
        
        return test_x, test_y


In [ ]:
def normalizing(X, y):

  print(y.shape)
  y_one_hot =  np.array([tf.one_hot(item, depth=3).numpy() for item in y])
  print(y_one_hot.shape)
  X_normal = X/255
  return X_normal, y_one_hot

In [ ]:
x_input, y_mask = normalizing(x_input, y_mask)

### 2. Display Images

In [ ]:
def display_image(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    if len(display_list[i].shape) == 3:
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        #plt.axis('off')
    else:
        plt.imshow(display_list[i])
  plt.show()

def display(list_train, list_mask):
  for idx, img_train in enumerate(list_train):
    sample_image, sample_mask = list_train[idx], list_mask[idx]
    sample_image = sample_image[..., :4]
    display_image([sample_image, sample_mask])

In [ ]:
display(x_input[:6], y_mask[:6])

### 3. Compiling the model

In [ ]:
def unet_2d(input_shape, num_classes):

    # Define the input layer
    inputs = Input(input_shape)

    # Downsample layers
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)

    # Upsample layers
    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv7)

    # Define the model
    model = Model(inputs=[inputs], outputs=[output])

    return model

In [ ]:
model = unet_2d(input_shape=(256, 256, 5), num_classes=3)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  2944        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

### 4. Splitting dataset

In [ ]:
# Split the data into training and test sets, with 80% for training and 20% for testing
x_train, x_test, y_train, y_test = train_test_split(x_input, y_mask, test_size=0.2, random_state=42)

# Split the training data into training and validation sets, with 75% for training and 25% for validation
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

# Print the shapes of the resulting datasets
print("Training data shape:", x_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation data shape:", x_val.shape)
print("Validation labels shape:", y_val.shape)
print("Test data shape:", x_test.shape)
print("Test labels shape:", y_test.shape)

### 5. Model training

In [ ]:
tile_size = 256
step_size = 256
saving_path = 'experiment_3'
training_dates = '17 images skikitsplit'
validation_dates = '17 images skikitsplit'
testing_dates = '17 images skikitsplit'

In [ ]:
def execute_training(count):
  print(f'Start training number {count}')
  model.compile(optimizer='adam',
                loss=categorical_crossentropy,
                metrics=['accuracy']) # ??? alternatives

  early_stop = EarlyStopping(monitor='accuracy', patience=5) 

  model_history = model.fit(x=x_train, y=y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stop])

  # saving model
  model_name = f'{tile_size}_{step_size}_run_{count}'
  model.save(f'../models/{saving_path}/model_{model_name}.h5')

  # saving model history
  with open(f'../models/{saving_path}/history_{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(model_history.history, file_pi)

  # making predictions
  predictions = model.predict(x_test)

  # calculating metrics
  metrics = EvaluationMetrics(x_train, x_val, x_test, y_train, y_val, y_test, predictions, training_dates, validation_dates, testing_dates, tile_size, step_size, count)
  print(f'jacard index: {metrics.jacard}')
  # saving metrics
  with open(f'../metrics/{saving_path}/history_{model_name}.pkl', 'wb') as file_pi:
      pickle.dump(metrics, file_pi)

  return metrics, model_history

In [ ]:
metrics, model_history = execute_training(0)

Start training number 0
Epoch 1/100
24/24 [==============================] - 31s 538ms/step - loss: 0.6058 - accuracy: 0.7843 - val_loss: 27.4159 - val_accuracy: 0.8723
Epoch 2/100
24/24 [==============================] - 6s 259ms/step - loss: 0.1863 - accuracy: 0.9432 - val_loss: 18.5538 - val_accuracy: 0.9680
Epoch 3/100
24/24 [==============================] - 6s 261ms/step - loss: 0.1154 - accuracy: 0.9677 - val_loss: 19.3050 - val_accuracy: 0.9706
Epoch 4/100
24/24 [==============================] - 6s 260ms/step - loss: 0.0930 - accuracy: 0.9715 - val_loss: 11.2852 - val_accuracy: 0.9732
Epoch 5/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0847 - accuracy: 0.9715 - val_loss: 10.6627 - val_accuracy: 0.9740
Epoch 6/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0728 - accuracy: 0.9736 - val_loss: 10.6931 - val_accuracy: 0.9723
Epoch 7/100
24/24 [==============================] - 6s 259ms/step - loss: 0.0777 - accuracy: 0.9718 - val_loss

statistics labels